<br><br>

## Práctica 6

# Sistema de recomendación en Spark


### Luis de la Ossa


Escuela Superior de Ingeniería Informática de Albacete


*Universidad de Castilla-La Mancha*

---

### Introducción

El objetivo de esta práctica es familiarizarse con la programación y tratamiento de datos en _Apache Spark_. Para ello, se implementará un algoritmo de recomendación basado en similaridad entre películas basado en las votaciones de los usuarios.

Hay que tener en cuenta que _Spark_ hace las operaciones cuando necesita los datos, y no antes. En ese caso, se reportan los errores, pero éstos pueden haberse producido en celdas anteriores aparentemente correctas. Con el fin de que podáis comprobar cada paso, se ha añadido un `collect()` comentado. 

__Nota__: El fin de este trabajo es esencialmente didáctico. Por tanto, no todos los pasos están optimizados. 

### Lectura de datos

Para esta práctica, al igual que en la anterior, usaremos los archivos `u.data` y `u.items`. Para que Spark los lea es necesario proporcionar el acceso, que dependerá de si los tenemos almacenados en local, o trabajamos con el entorno DSX de IBM. 

Las siguientes celdas almacenan la ruta de acceso a cada uno de los archivos. En caso de trabajar en DSX, han de ser generadas desde el propio entorno. En el caso de `u.item` como _StringIO_, y en el caso de `u.data` como _SparkSession_.

In [1]:

from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_74af1899931d4f94bb91ee5a4f761112(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_225e6d7e906e9d9577669d9e9514b01252b88460','domain': {'id': '1777dedc403547c98be0dcc0821187a2'},
            'password': 'w3NIn)2RB/FI-ylx'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

# Your data file was loaded into a StringIO object and you can process the data.
# Please read the documentation of pandas to learn more about your possibilities to load your data.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html
path_movies = get_object_storage_file_with_credentials_74af1899931d4f94bb91ee5a4f761112('SistemaRecomendacion', 'u.item')


In [2]:

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_74af1899931d4f94bb91ee5a4f761112(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', 'e67e4544bd794bb7bf3a337a94365c28')
    hconf.set(prefix + '.username', '796eadbf96a44e40a20ada8abfb937da')
    hconf.set(prefix + '.password', 'w3NIn)2RB/FI-ylx')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_74af1899931d4f94bb91ee5a4f761112(name)

# Please read the documentation of PySpark to learn more about the possibilities to load data files.
# PySpark documentation: https://spark.apache.org/docs/1.6.0/api/python/pyspark.sql.html#pyspark.sql.SQLContext
# The SQLContext object is already initalized for you.
# The following variable contains the path to your file on your Object Storage.
path_ratings = "swift://SistemaRecomendacion." + name + "/u.data"


A continuación, y puesto que la lista de películas solamente se utilizará en modo local, la almacenaremos en un _DataFrame_ Pandas.

In [3]:
import pandas as pd

names_mv = ['id', 'title', 'release', 'video', 'IMDb URL',
            'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 
            'Documentary', 'Drama', 'Fantasy',' Film-Noir', 'Horror', 'Musical', 'Mystery', 
            'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

df_movies = pd.read_csv(path_movies, sep='|', encoding='iso-8859-1', names=names_mv)

Los datos relativos a las votaciones los almacenaremos en un RDD.

In [4]:
dataRDD = sc.textFile(path_ratings)

---

### Algoritmo de recomendación basado en similaridad entre películas.

En esta parte implementaremos un algoritmo basado en similaridad entre películas. Posteriormente, se obtendrán las más similares a una dada. 

La técnica es parecida a la vista en clase. Dadas dos películas, su similaridad se obtiene a partir de las valoraciones que han hecho los usuarios __que hayan visto ambas__. Hemos de pensar (aunque habría que cambiar algunas cosas en el programa si en realidad fuera así) que el número de usuarios es muy grande. 

#### Ejercicio 1.

En primer lugar, vamos a convertir el RDD de texto en un RDD con tuplas `(user_id, (movie_id, rating))`, denominado `ratingsRDD`, en el que `user_id` y `movie_id` son enteros, y `rating`es float. Utilizar la función `map`.

In [5]:
ratingsRDD = dataRDD.map(lambda x: x.split('\t'))
ratingsRDD = ratingsRDD.map(lambda x: tuple([int(x[0]),tuple([int(x[1]),float(x[2])])]))

print(ratingsRDD.collect()[:3])
#[(196, (242, 3.0)), (186, (302, 3.0)), (22, (377, 1.0))]

[(196, (242, 3.0)), (186, (302, 3.0)), (22, (377, 1.0))]



---

De cara a diseñar el algoritmo, el principal factor a tener en cuenta es que se van a comparar pares de películas, y para ello solamente se considerán las valoraciones de usuarios que hayan visto ambas. Por ejemplo, si dos películas `movie_A`y `movie_B` han sido valoradas por cinco usuarios

$$
movie\_A = [ 0, 2, 4, 0, 1] \quad movie\_B = [1 ,3, 5, 0, 0], 
$$

solamente se se utilizarán las valoraciones de los usuarios 2 y 3, que han visto ambas películas. Visto de otro modo, más aproximado a cómo se procederá aquí, el usuario 1, por ejemplo, no se tendrá en cuenta para calcular la distancia entre la película `movie_A` y ninguna otra. Por ello en primer lugar se van a generar los pares de películas vistas por un mismo usuario. 

En el apartado anterior se han generado tuplas del tipo `(user_id, (movie_id, rating))`. Si se hace un _inner join_ de `ratingsRDD` con él mismo, cada entrada del primer RDD se unirá con las entradas del segundo RDD con las que comparta clave (`user_id`), generando tuplas del tipo:  `(user_id, ((movie_id1, rating1), (movie_id2, rating2))`.



#### Ejercicio 2. 

Hacer un join del RDD `ratingsRDD` y almacenar el resultado en otro denominado `join_ratingsRDD`.

In [6]:
join_ratingsRDD = ratingsRDD.join(ratingsRDD)

join_ratingsRDD.collect()[:3]
#[(512, ((265, 4.0), (265, 4.0))),
# (512, ((265, 4.0), (23, 4.0))),
# (512, ((265, 4.0), (1, 4.0)))]

[(512, ((265, 4.0), (265, 4.0))),
 (512, ((265, 4.0), (23, 4.0))),
 (512, ((265, 4.0), (1, 4.0)))]


---

En este punto, cada par de películas vistas por un usuario aparece dos veces en `join_ratingsRDD`, ya que aparecerían en orden inverso. Es decir, por cada entrada `(user_id, ((movie_id1, rating1), (movie_id2, rating2))` tendríamos otra como `(user_id, ((movie_id2, rating2), (movie_id1, rating1))`. Para eliminar los duplicados, vamos a dejar solamente aquellas entradas en las que el índice de la primera película `movie_id1`, sea menor que el de la segunda `movie_id2`.

#### Ejercicio 3.

Implementar una función denominada `filter_duplicates` que reciba una tupla del tipo `((movie_id1, rating1), (movie_id2, rating2))` y devuelva `True` cuando `movie_id1 < movie_id2`, y `False` en caso contrario. 

In [7]:
def filter_duplicates(ratings):
    return ratings[0][0] < ratings[1][0]


---

#### Ejercicio 4.

Utilizar la función anterior para eliminar los duplicados de `join_ratingsRDD`, mediante `filter`. 

__Nota__: Hay que tener en cuenta que cada entrada de `join_ratingsRDD` es una tupla `(user_id, ((movie_id1, rating1), (movie_id2, rating2))`. Por tanto, a la función `filter_duplicates` hay que pasarle el segundo de los componentes de la tupla, ya que el primero es `user_id`.

In [8]:
join_ratingsRDD = join_ratingsRDD.filter(lambda x: filter_duplicates(x[1]))

join_ratingsRDD.collect()[:3]
#[(512, ((265, 4.0), (318, 5.0))),
# (512, ((265, 4.0), (1459, 4.0))),
# (512, ((265, 4.0), (313, 3.0)))]

[(512, ((265, 4.0), (318, 5.0))),
 (512, ((265, 4.0), (1459, 4.0))),
 (512, ((265, 4.0), (313, 3.0)))]


---

En este punto, se dispone de entradas `(user_id, ((movie_id1, rating1), (movie_id2, rating2))`. De cara a computar la similaridad entre películas, se incluirán en los vectores correspondientes a las películas `movie_id1` y `movie_id2` las valoraciones correspondientes al usuario `user_id`.  Sin embargo, el usuario en si es irrelevante. Por otra parte, de cara a hacer los cálculos de similaridad, lo importante es buscar los pares de películas.

#### Ejercicio 5.

Transformar el RDD `join_ratingsRDD` en el que las entradas son del tipo,  `(user_id, ((movie_id1, rating1), (movie_id2, rating2))`, en un RDD pareado denominado `movie_pairsRDD`, en el que las entradas sean de tipo `((movie_id1, movie_id2),(rating1, rating2))`. Utilizar la función `map`.

In [9]:
movie_pairsRDD = join_ratingsRDD.map(lambda x: tuple([tuple([x[1][0][0],x[1][1][0]]),tuple([x[1][0][1],x[1][1][1]])]))

movie_pairsRDD.collect()[:3]
#[((265, 318), (4.0, 5.0)), ((265, 1459), (4.0, 4.0)), ((265, 313), (4.0, 3.0))]

[((265, 318), (4.0, 5.0)), ((265, 1459), (4.0, 4.0)), ((265, 313), (4.0, 3.0))]

Para cada poder calcular la similaridad entre un par de películas, han de obtenerse las valoraciones echas por cada usuario que haya visto ambas, es decir, buscar todas las tuplas `(movie_id1, movie_id2)`, y agrupar los pares `(rating1, rating2)` de cada una de ellas.

---

#### Ejercicio 6.

Agrupar las valoraciones existentes en el RDD `movie_pairsRDD` para cada par `(movie_id1, movie_id2)`, generando un RDD pareado denominado `movie_pairs_ratingsRDD` del tipo `(movie_id1, movie_id2): [(rating1, rating2), (rating1, rating2), ...] `.  Utilizar para ello `groupByKey()`.

In [10]:
movie_pairs_ratingsRDD = movie_pairsRDD.groupByKey()

movie_pairs_ratingsRDD.collectAsMap()[(366,854)].data   # Obtiene los ratings correspondientes a la entrada (366,854) (tres en total).
#[(3.0, 4.0), (2.0, 1.0), (3.0, 1.0)]

[(3.0, 1.0), (2.0, 1.0), (3.0, 4.0)]


---

### Cálculo de la distancia coseno. Opción 1.

A continuación, y aunque en este contexto no es necesario, vamos a transformar la lista de valoraciones para cada par de películas, que tenemos almacenada con formato  `[(3.0, 4.0), (2.0, 1.0), (3.0, 1.0)]` a un formato en el que la lista de valoraciones de cada película se represente como una dos tupla de tipo `((3.0, 2.0, 3.0), (4.0, 1.0, 1.0))`, que podrán ser convertidas posteriormente en vectores `Numpy` para el cálculo de la similaridad. El resultado lo guardaremos en la variable `movie_pairs_ratings2VRDD`.

__Nota__: Este paso no es estrictamente necesario. Incluso en este caso puede compensar calcular directamente la distancia coseno con una versión iterativa de la misma. Pero en otras situaciones, sí que puede serlo.

In [11]:
movie_pairs_ratings2VRDD = movie_pairs_ratingsRDD.mapValues(lambda ratings: list(zip(*ratings.data)))
movie_pairs_ratingsRDD.collectAsMap()[(197, 1097)]
#[(4.0, 3.0, 5.0, 3.0, 4.0, 2.0, 5.0), (5.0, 2.0, 4.0, 3.0, 4.0, 3.0, 4.0)]

Ahora podemos definir la función similaridad coseno, de modo que reciba una lista con dos tuplas (los dos vectores), los transforme en vectores, y devuelva sus distancias, así como el número de elementos que tienen.

In [12]:
import numpy as np

def cos_sim_v(ratings):
    # Convierte a arrays
    ratings_movie_0 = np.asarray(ratings[0])
    ratings_movie_1 = np.asarray(ratings[1])
    # Calcula la similaridad
    num = ratings_movie_0.dot(ratings_movie_1)
    den = np.sqrt(ratings_movie_0.dot(ratings_movie_0))*np.sqrt(ratings_movie_1.dot(ratings_movie_1))
    return num/den, len(ratings[0])

cos_sim_v([(4.0, 3.0, 5.0, 3.0, 4.0, 2.0, 5.0), (5.0, 2.0, 4.0, 3.0, 4.0, 3.0, 4.0)])
#(0.97587290935995985, 7)

(0.97587290935995985, 7)

---

#### Ejercicio 7

Calcular la similaridad coseno para cada par de películas de almacenadas en `movie_pairs_ratings2VRDD` y almacenar el resultado en `movie_pair_similaritiesRDD`. Hacer persistente el resultado llamando al método `cache()`. El resultado incluye la similaridad, así como el número de personas que han valorado ambas películas (es lo que devuelve la función `cos_sim_v`).

In [13]:
movie_pair_similaritiesRDD = movie_pairs_ratings2VRDD.mapValues(lambda x: cos_sim_v(x)).cache()

movie_pair_similaritiesRDD.collect()[:3]
#[((197, 1097), (0.97587290935995985, 7)),
# ((42, 364), (0.90934865603988357, 18)),
# ((273, 617), (0.96529535990071047, 7))]

[((197, 1097), (0.97587290935995985, 7)),
 ((773, 1409), (1.0, 1)),
 ((273, 617), (0.96529535990071047, 7))]


---

### Cálculo de la distancia coseno. Opción 2.

En el caso de que los vectores de valoraciones comunes para las películas, almacenados en `movie_pairs_ratingsRDD` con formato `[(3.0, 4.0), (2.0, 1.0), (3.0, 1.0)]` sean excesivamente largos, puede no ser adecuada la vectorización. En ese caso, puede calcularse directamente la similaridad coseno con un algoritmo tradicional.

In [14]:
from math import sqrt

def cos_sim(ratings):
    num_ratings = 0
    sum_1 = sum_2= sum_12 = 0    
    for rating1, rating2 in ratings:
        sum_1 += rating1 * rating1
        sum_2 += rating2 * rating2
        sum_12 += rating1 * rating2
        num_ratings += 1
    
    num = sum_12
    den = sqrt(sum_1) * sqrt(sum_2)

    return num / float(den), num_ratings

cos_sim([(3.0, 4.0), (2.0, 1.0), (3.0, 1.0)])
#(0.8542821429703302, 3)

(0.8542821429703302, 3)

---

#### Ejercicio 8

Calcular la similaridad coseno (ahora con el método `cos_sim`) para cada par de películas de almacenadas en `movie_pairs_ratingsRDD` (el original que contiene 2-tuplas) y almacenar el resultado en `movie_pair_similaritiesRDD`. Hacer persistente el resultado llamando al método `cache()`. El resultado incluye la similaridad, así como el número de personas que han valorado ambas películas (es lo que devuelve la función `cos_sim`.

In [15]:
movie_pair_similaritiesRDD = movie_pairs_ratingsRDD.mapValues(lambda x: cos_sim(x)).cache()

movie_pair_similaritiesRDD.collect()[:3]
#[((197, 1097), (0.9758729093599599, 7)),
# ((773, 1409), (1.0, 1)),
# ((273, 617), (0.9652953599007105, 7))]

[((197, 1097), (0.9758729093599599, 7)),
 ((42, 364), (0.9093486560398836, 18)),
 ((273, 617), (0.9652953599007105, 7))]

## Películas similares a una dada.

A continuación, se van a generar las películas similares a una dada (la 10). Solamente se considerarán aquellas cuya similaridad esté por encima de 0.97 y, además, hayan sido votadas por al menos 20 personas que votaron la película original. 

In [16]:
min_sim = 0.97
min_common = 20
movie_id = 10

---

#### Ejercicio 9

Filtrar los resultados de interés. Es decir, lo que que cumplan los criterios descritos anteriormente y almacenarlos en un RDD denominado `similar_moviesRDD`, es decir, que la primera o la segunda película sea `movie_id`, que la similaridad esté por encima de 0.97, y que el número de personas que han visto ambas sea al menos 50.  Utilizar la función `filter`.

In [17]:
similar_moviesRDD = movie_pair_similaritiesRDD.filter(lambda x: (x[0][0] == movie_id or x[0][1] == movie_id) and x[1][0] > min_sim and x[1][1] > min_common)

similar_moviesRDD.collect()[:3]
#[((10, 792), (0.978909167336141, 24)),
# ((10, 528), (0.9748821327187926, 25)),
# ((10, 387), (0.9762910550634251, 21))]

[((10, 792), (0.978909167336141, 24)),
 ((10, 528), (0.9748821327187926, 25)),
 ((10, 527), (0.9723706468984957, 29))]


---

#### Ejercicio 10

Ordenar los resultados de `similar_moviesRDD` por similaridad. Utilizar para ello `sortBy`. Devolver los 10 primeros elementos del RDD resultante con `take` y almacenarlos en la lista `results`.

In [18]:
results = similar_moviesRDD.sortBy(lambda x: x[1][0]).take(10)[::-1]

results[:3]
#[((10, 558), (0.9885451117519884, 21)),
# ((10, 474), (0.9801007017199401, 34)),
# ((10, 223), (0.9796226612103792, 27))]

[((10, 792), (0.978909167336141, 24)),
 ((10, 709), (0.9788941627901632, 22)),
 ((10, 387), (0.9762910550634251, 21))]

---

#### Muestra los resultados

A continuación, se muestran los resultados obtenidos.

In [19]:
for result in results:
    print(result)

((10, 792), (0.978909167336141, 24))
((10, 709), (0.9788941627901632, 22))
((10, 387), (0.9762910550634251, 21))
((10, 59), (0.9749784735609486, 21))
((10, 528), (0.9748821327187926, 25))
((10, 527), (0.9723706468984957, 29))
((10, 169), (0.9718073257951023, 22))
((10, 190), (0.9714818546315672, 39))
((10, 429), (0.9708688309717662, 21))
((10, 89), (0.9701678111320445, 45))


In [20]:
print("Las 10 películas más parecidas a " + df_movies.iloc[movie_id]['title']+"\n\n")

for result in results:
    (pair, sim) = result
    sim_movie_id = pair[1]
    print(df_movies.iloc[sim_movie_id]['title'] + "\n\t\tSimilaridad: %.2f \t Votada en común:%d\n" % (sim[0],sim[1]))

Las 10 películas más parecidas a Seven (Se7en) (1995)


Crooklyn (1994)
		Similaridad: 0.98 	 Votada en común:24

Better Off Dead... (1985)
		Similaridad: 0.98 	 Votada en común:22

Beverly Hills Cop III (1994)
		Similaridad: 0.98 	 Votada en común:21

Three Colors: Blue (1993)
		Similaridad: 0.97 	 Votada en común:21

My Life as a Dog (Mitt liv som hund) (1985)
		Similaridad: 0.97 	 Votada en común:25

Killing Fields, The (1984)
		Similaridad: 0.97 	 Votada en común:29

Cinema Paradiso (1988)
		Similaridad: 0.97 	 Votada en común:22

Amadeus (1984)
		Similaridad: 0.97 	 Votada en común:39

Duck Soup (1933)
		Similaridad: 0.97 	 Votada en común:21

So I Married an Axe Murderer (1993)
		Similaridad: 0.97 	 Votada en común:45



---

## Librería MLib: ALS (Alternative Least Squares)

La librería _MLib_ contiene algoritmos de aprendizaje automáticos implementados para ejecutarse sobre la plataforma _Spark_. 

Como ejemplo, vamos entrenar un sistema de recomendación basado en _Alternative Least Squares_. Como se vio en el tema, este algoritmo optimiza tanto los parámetros que caracterizan las películas ($X$) como los que caracterizan a los usuarios $\theta$. Es decir, hace una factorización de matrices.

In [21]:
from pyspark.mllib.recommendation import ALS, Rating

---

#### Ejercicio 11.

Transformar los datos del RDD `dataRDD` en otro RDD de objetos `Rating`. Cada uno de ellos se crea como `Rating(user_id, movie_id,rating)`.

In [22]:
ratingsRDD = dataRDD.map(lambda x: x.split('\t'))
ratingsRDD = ratingsRDD.map(lambda x: Rating(x[0],x[1],x[2]))

---

#### Ejercicio 12.

Entrenar el modelo, con los parámetros proporcionados (mirar documentación). 

In [23]:
rank = 10
n_iterations = 6
model = ALS.train(ratingsRDD, rank, iterations=n_iterations)


---

#### Ejercicio 13. 

Obtener las valoraciones para el usuario 20 y almacenarlas en `user_ratingsRDD`.

In [24]:
user_id = 20
print("\nValoraciones para el usuario " + str(user_id) + ": \n")

# Filtra por usuario
dataRDD2 = dataRDD.map(lambda x: x.split('\t'))
user_ratingsRDD = dataRDD2.filter(lambda x: int(x[0]) == user_id)

print("El usuario ha votado %d películas \n\n" %(user_ratingsRDD.count()))

# Las imprime
for rating in user_ratingsRDD.collect():
    print(df_movies.iloc[int(rating[1])]['title']+": " + str(rating[2]))


Valoraciones para el usuario 20: 

El usuario ha votado 48 películas 


Evita (1996): 1
This Is Spinal Tap (1984): 2
Usual Suspects, The (1995): 2
Good, The Bad and The Ugly, The (1966): 2
Maya Lin: A Strong Clear Vision (1994): 4
Godfather: Part II, The (1974): 3
Princess Bride, The (1987): 3
Conan the Barbarian (1981): 4
Terminator, The (1984): 3
Thinner (1996): 3
GoldenEye (1995): 3
Four Weddings and a Funeral (1994): 1
Crow: City of Angels, The (1996): 4
Big Squeeze, The (1996): 3
First Kid (1996): 1
Terminator 2: Judgment Day (1991): 3
Cable Guy, The (1996): 3
Pillow Book, The (1995): 4
Wolf (1994): 4
Lawnmower Man, The (1992): 2
Snow White and the Seven Dwarfs (1937): 3
Sleepless in Seattle (1993): 5
Cat on a Hot Tin Roof (1958): 3
Wild Bunch, The (1969): 4
Whole Wide World, The (1996): 1
Children of the Corn: The Gathering (1996): 2
GoodFellas (1990): 4
Aladdin (1992): 2
If Lucy Fell (1996): 1
Paradise Road (1997): 4
M*A*S*H (1970): 4
Brazil (1985): 4
Mrs. Winterbourne (1996): 

---

#### Ejercicio 14.

Obtener e imprimir las 10 mejores recomendaciones para el usuario (a partir del objeto `model` creado anteriormente.

In [97]:
print("\nMejores 10 recomendaciones \n ")

recommendations = model.recommendProducts(user_ratingsRDD).take(10)

for recommendation in recommendations:
    print(df_movies.iloc[int(recommendation[1])]['title'] + "\t Score " + str(recommendation[2]))


Mejores 10 recomendaciones 
 


TypeError: recommendProducts() missing 1 required positional argument: 'num'